In [2]:
# Introduction :

In [135]:
#

import re
from tqdm import tqdm
from ast import literal_eval
import os
import vcf
import gzip
from tqdm import tqdm
from collections import Counter
import pandas as pd
import os
import numpy as np
import itertools
import re
import matplotlib.pyplot as plt
import numpy as np
from pysam import VariantFile
import json
import pysam

In [3]:
# Analysing the Microsatellite Denovo parent of origin.


# 1. Question one is to see the distribution of number of mDNMs from each trio.
# 2. Question 2 is to check the distribution between counts of 'P' & 'M'.
#

In [110]:
# Dictionary which contains the Union of Phased mDNMs and frequency in all individuals.

Phased_mDMNs = {}

In [111]:

def Phased_files(infile):

    with gzip.open(f'../../Analysis_KT/SNP_Denovo/SNPs_Intermediate_Files/SNPs_AssociatedDenovoSTR_lens/{infile}', 'rt') as mDNMs :
            mDNMs.readline()

            for Dregion in mDNMs:
                Dregion = Dregion.strip().split('\t')
                if int(Dregion[-1]) == 0: continue
                if len(Dregion[3]) == 1: continue
                region = f'{Dregion[0]}:{Dregion[1]}-{Dregion[2]}'
                Denovo_alleles = Dregion[7].split('|')
                phased_alleles_info = Dregion[-2].split(':')


                # Checking if phased alleles are part of Denovo alleles.
                usefull_allele_info = []
                test = [usefull_allele_info.append(1) for x in phased_alleles_info  if len(set(Denovo_alleles) - set(x.split('/'))) == 0]
                
                # Checking regions where both mDNMs got phased and updating to dictionary- where we get the union of all mDNMs regions and their frequency among the samples.
                if len(usefull_allele_info) != 0:
                    if region not in Phased_mDMNs: Phased_mDMNs[region] = 1
                    else: Phased_mDMNs[region] += 1


In [112]:
# itterating the files.

read_directory = '../../Analysis_KT/SNP_Denovo/SNPs_Intermediate_Files/SNPs_AssociatedDenovoSTR_lens/'

for file in tqdm(os.listdir(read_directory)):
    
    # checking if the file has any info or not.
    if file.endswith('_Phased_mDNMs.tsv.gz') :
        Phased_files(file)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 597/597 [00:01<00:00, 356.23it/s]


In [113]:
len(Phased_mDMNs)

41710

# RESULT : Here we see 1.6 lakhs STR regions which got phased.
### We see without Monomers, 41,710 thousand STR regions ----- why monomers why ... ?

In [136]:
# list of list containing the each sample "Maternal and Paternal" region counts. Out of 8,311.

Paternal_Maternal_counts = [[], [], []] # [Number of mDNMs] [P alleles] [M alleles]

In [137]:
# Function : Checking if phased alleles are part of Denovo alleles.

def checking_phased_alleles(Dregion, phased_alleles_info, Denovo_alleles):
    
    usefull_allele_info = []
    check = [usefull_allele_info.append(i) for i,x in enumerate(phased_alleles_info)  if len(set(Denovo_alleles) - set(x.split('/'))) == 0]

    # Checking regions where both mDNMs got phased and updating to dictionary- where we get the union of all mDNMs regions and their frequency among the samples.
    if len(usefull_allele_info) > 0:
        return usefull_allele_info[0]


In [138]:
# Function : returning the Denovo allele and its parent of origin.
# ([['30', 'G'], ['36', 'A']], [['G', 'M'], ['A', 'P']], '36')
def Denovo_allele_origin(informative_Dallele_info, informative_Snp_allele_info, Denovo_allele, Dregion):
    
    Dallele_split = re.split(r',|:', informative_Dallele_info)  # check the format after splitting the alleles and snp info
    Snp_PofO = re.split(r';|:|,', informative_Snp_allele_info) # check the format
    
    Dallele_info = [Dallele_split[0:2], Dallele_split[3:5]]         # check the '_12' ******************
    Snp_PofO = [Snp_PofO[1:3], Snp_PofO[3:]]                        # check the '_A' *******************
    
    # Creating the dictionary to fetch the info stright.
    SNP_allele_info = {Snp_PofO[0][0]: Snp_PofO[0][1], Snp_PofO[1][0]: Snp_PofO[1][1]}
    Denovo_allele_pofo = {Dallele_info[0][0]: Dallele_info[0][1], Dallele_info[1][0]: Dallele_info[1][1]}
    
    # PofO = None
    # Here we get the Denovo allele has come from the Paternal or Maternal side.
    if Denovo_allele in Denovo_allele_pofo: 
        PofO = SNP_allele_info[Denovo_allele_pofo[Denovo_allele]]
        return PofO #, Dallele_info, Snp_PofO, SNP_allele_info, Denovo_allele_pofo, Denovo_allele


In [144]:
# A very 1st question :

#-----> one:- Number of mDNMs in maternal and paternal side.
t = []

def Phased_files1(infile):
    #
    i = 0
    j = 0
    Maternal_Paternal_counts = []
    
    with open(f'../../Analysis_KT/SNP_Denovo/SNPs_Intermediate_Files/SNPs_AssociatedDenovoSTR_lens/{infile}') as mDNMs :
            mDNMs.readline()
            
            for i,Dregion in enumerate(mDNMs):
                Dregion = Dregion.strip().split('\t')
                if int(Dregion[-1]) == 0: continue
                if len(Dregion[3]) == 1: continue
                j += 1
                #
                Denovo_alleles = Dregion[7].split('|')
                phased_alleles_info = Dregion[-2].split(':')
                
                # Passing the arguments to see if the phased alleles are part of Denovo alleles.
                usefull_index = checking_phased_alleles(Dregion, phased_alleles_info, Denovo_alleles)
                
                #
                if usefull_index != None:
                    informative_Dallele_info = Dregion[-3].split(';')[usefull_index].strip()
                    informative_Snp_allele_info = Dregion[-5].split('-')[usefull_index].strip()
                    Denovo_allele = Dregion[8]
                    
                    # Here we look into the phased Denovo_allele and to count the M or P contribution.
                    temp = Denovo_allele_origin(informative_Dallele_info, informative_Snp_allele_info, Denovo_allele,Dregion)
                    Maternal_Paternal_counts.append(temp)
                    # print(temp)
                
                else: t.append(1)
                
                
                            
                    
    Trio_phased_info = Counter(Maternal_Paternal_counts)
    # if int(Trio_phased_info['P']) > int(Trio_phased_info['M']):
    print(Trio_phased_info, len(t), i,j)
        
Phased_files1('TRIO0001_test.tsv')

Counter() 375 1260 375


In [120]:
# itterating the files.

# read_directory = '../../Analysis_KT/SNP_Denovo/SNPs_Intermediate_Files/SNPs_AssociatedDenovoSTR_lens/'

# for file in tqdm(os.listdir(read_directory)):
    
#     # checking if the file has any info or not.
#     if file.endswith('_Phased_mDNMs.tsv.gz') :
#         Phased_files1(file)